In [1]:
%load_ext autoreload  
%autoreload 2 

In [2]:
from src.subset_classifier import *
import pandas as pd
import sqlite3
import sqlalchemy 
import json
from datasets import load_dataset
from collections import Counter
from pydantic.dataclasses import dataclass
from transformers import AutoTokenizer

/Users/garylai/Dev/subset-active-learning/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DB_PATH = "./subset_selection/sst_results.db"
df = get_df_from_db(DB_PATH)

In [4]:
subset_unique_counts = get_subset_unique_counts(df); print(subset_unique_counts)

Counter({96: 3164, 93: 1815, 94: 1793, 95: 876, 92: 661, 97: 262})


In [5]:
optimal_subset_data_indices = get_optimal_subset_data_indices(df)

In [6]:
sst2 = load_dataset("sst")
data_pool = sst2["train"].shuffle(seed=0).select(range(1000))

100%|██████████| 3/3 [00:00<00:00, 368.96it/s]


In [7]:
config = OptimalSubsetClassifierConfig(max_length=66, debug=False, model_name="google/electra-small-discriminator", batch_size=8)

In [8]:
ds = preprocess(data_pool, config, optimal_subset_data_indices)

In [9]:
type(optimal_subset_data_indices)

set

In [10]:
data_pool_indices = set(range(1000))
non_optimal_subset_data_indices = list(data_pool_indices - optimal_subset_data_indices)
train_indices, valid_indices, test_indices = create_stratefied_split(
    list(optimal_subset_data_indices), list(non_optimal_subset_data_indices), split_points=(0.8, 0.9)
)

INFO:src.subset_classifier:dataset      num positive examples    num negative examples
---------  -----------------------  -----------------------
train                           77                      723
valid                            9                       91
test                            10                       90


In [11]:
train_ds = ds.select(train_indices)
valid_ds = ds.select(valid_indices)
test_ds = ds.select(test_indices)

In [12]:
len(train_ds), len(valid_ds), len(test_ds)

(800, 100, 100)